## Skorch basic classification example
#### from skorch docs/readme - https://github.com/skorch-dev/skorch

In [ ]:
import subprocess

# Installation on Google Colab
try:
    import google.colab
    subprocess.run(['python', '-m', 'pip', 'install', 'skorch' , 'torch'])
except ImportError:
    pass

In [4]:
import numpy as np
from sklearn.datasets import make_classification
from torch import nn
from skorch import NeuralNetClassifier

X, y = make_classification(1000, 20, n_informative=10, random_state=0)
X = X.astype(np.float32)
y = y.astype(np.int64)

class MyModule(nn.Module):
    def __init__(self, num_units=10, nonlin=nn.ReLU()):
        super().__init__()

        self.dense0 = nn.Linear(20, num_units)
        self.nonlin = nonlin
        self.dropout = nn.Dropout(0.5)
        self.dense1 = nn.Linear(num_units, num_units)
        self.output = nn.Linear(num_units, 2)
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = self.dropout(X)
        X = self.nonlin(self.dense1(X))
        X = self.softmax(self.output(X))
        return X

net = NeuralNetClassifier(
    MyModule,
    max_epochs=10,
    lr=0.1,
    # Shuffle training data on each epoch
    iterator_train__shuffle=True,
)

net.fit(X, y)
y_proba = net.predict_proba(X)

  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7548       0.5000        0.7079  0.0136
      2        0.6990       0.5350        0.6864  0.0193
      3        0.6823       0.5400        0.6800  0.0191
      4        0.6727       0.5900        0.6739  0.0208
      5        0.6652       0.6000        0.6676  0.0130
      6        0.6590       0.6400        0.6577  0.0177
      7        0.6498       0.6450        0.6507  0.0212
      8        0.6379       0.6750        0.6351  0.0165
      9        0.6248       0.6850        0.6225  0.0183
     10        0.6128       0.7050        0.6135  0.0164


## Using sklearn pipeline

In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

pipe = Pipeline([
    ('scale', StandardScaler()),
    ('net', net),
])

pipe.fit(X, y)
y_proba = pipe.predict_proba(X)

Re-initializing module.
Re-initializing criterion.
Re-initializing optimizer.
  epoch    train_loss    valid_acc    valid_loss     dur
-------  ------------  -----------  ------------  ------
      1        0.7128       0.5000        0.7006  0.0215
      2        0.6963       0.5150        0.6934  0.0258
      3        0.6903       0.5550        0.6901  0.0225
      4        0.6873       0.5500        0.6885  0.0216
      5        0.6857       0.5500        0.6868  0.0261
      6        0.6830       0.5600        0.6851  0.0205
      7        0.6805       0.5700        0.6828  0.0237
      8        0.6775       0.5500        0.6807  0.0233
      9        0.6694       0.5450        0.6774  0.0221
     10        0.6714       0.5600        0.6740  0.0223


## Using sklearn grid search 

In [6]:
from sklearn.model_selection import GridSearchCV

# deactivate skorch-internal train-valid split and verbose logging
net.set_params(train_split=False, verbose=0)
params = {
    'lr': [0.01, 0.02],
    'max_epochs': [10, 20],
    'module__num_units': [10, 20],
}
gs = GridSearchCV(net, params, refit=False, cv=3, scoring='accuracy', verbose=2)

gs.fit(X, y)
print("best score: {:.3f}, best params: {}".format(gs.best_score_, gs.best_params_))

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV] END .......lr=0.01, max_epochs=10, module__num_units=10; total time=   0.1s
[CV] END .......lr=0.01, max_epochs=10, module__num_units=10; total time=   0.1s
[CV] END .......lr=0.01, max_epochs=10, module__num_units=10; total time=   0.1s
[CV] END .......lr=0.01, max_epochs=10, module__num_units=20; total time=   0.1s
[CV] END .......lr=0.01, max_epochs=10, module__num_units=20; total time=   0.1s
[CV] END .......lr=0.01, max_epochs=10, module__num_units=20; total time=   0.1s
[CV] END .......lr=0.01, max_epochs=20, module__num_units=10; total time=   0.2s
[CV] END .......lr=0.01, max_epochs=20, module__num_units=10; total time=   0.2s
[CV] END .......lr=0.01, max_epochs=20, module__num_units=10; total time=   0.2s
[CV] END .......lr=0.01, max_epochs=20, module__num_units=20; total time=   0.2s
[CV] END .......lr=0.01, max_epochs=20, module__num_units=20; total time=   0.2s
[CV] END .......lr=0.01, max_epochs=20, module__n